In [112]:

import os, json
import numpy as np, pandas as pd

from pprint import pprint
from datetime import datetime as dt


In [113]:

csv_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_bbb'


In [114]:

print(csv_location)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_json_bbb

2023-05-02


In [115]:

folder_path = csv_location

# Get a list of all files and directories in the folder
all_files_and_dirs = os.listdir(folder_path)

# Filter the list to include only files (exclude directories)
files_only = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Print total # of files
print(len(files_only))

# Print names of first few files 
pprint(files_only[:5])


24
['2023_10_Sunrisers Hyderabad vs Lucknow Super Giants.csv',
 '2023_11_Rajasthan Royals vs Delhi Capitals.csv',
 '2023_12_Mumbai Indians vs Chennai Super Kings.csv',
 '2023_13_Gujarat Titans vs Kolkata Knight Riders.csv',
 '2023_14_Punjab Kings vs Sunrisers Hyderabad.csv']


In [116]:

files_list = files_only
total_files = len(files_list)

#print(np.arange(total_files))
print(total_files)


24


In [117]:

#All file numbers > 1359475 are #2023 IPL onwards
#All file numbers > 1304047 are #2022 IPL onwards


In [118]:

# initialize an empty DataFrame to store all the data
master_df = pd.DataFrame()

# iterate through each file in the directory
for file_name in os.listdir(csv_location):
    if file_name.endswith('.csv'):
        # read the CSV file into a DataFrame
        file_path = os.path.join(csv_location, file_name)
        df = pd.read_csv(file_path)
        
        # append the data to the master DataFrame
        master_df = master_df.append(df)

# print the master DataFrame
print(master_df.shape)


(5859, 20)


In [119]:

print(master_df.columns.values.tolist())


['inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs']


In [120]:

display(master_df.head(3))


,inn,bat_tm,batnum,bat,ball,over_num,bow,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,m.grnd,m.date,ss,m.type,m.num,m.vs
0,1,Sunrisers Hyderabad,1,Anmolpreet Singh,0,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
1,1,Sunrisers Hyderabad,2,MA Agarwal,1,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
2,1,Sunrisers Hyderabad,1,Anmolpreet Singh,2,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants


In [121]:

master_df.describe()


,inn,batnum,ball,over_num,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,ss,m.num
count,5859.000000,5859.000000,5859.000000,5859.000000,5859.000000,5859.000000,0.0,5859.000000,5859.000000,5859.000000,5859.000000,5859.0,5859.000000
mean,1.486431,3.834101,2.653183,9.320191,1.365591,0.052398,NaN,0.957331,0.308756,0.506400,0.184844,2023.0,12.568868
std,0.499859,2.240444,1.843542,5.712608,1.749160,0.222847,NaN,0.202128,0.462020,0.500002,0.388204,0.0,6.918007
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,2023.0,1.000000
25%,1.000000,2.000000,1.000000,4.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,2023.0,7.000000
50%,1.000000,3.000000,3.000000,9.000000,1.000000,0.000000,NaN,1.000000,0.000000,1.000000,0.000000,2023.0,13.000000
75%,2.000000,5.000000,4.000000,14.000000,1.000000,0.000000,NaN,1.000000,1.000000,1.000000,0.000000,2023.0,19.000000
max,2.000000,11.000000,10.000000,19.000000,6.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,2023.0,24.000000


## Batting stats 

In [122]:

bat_grouped_df = master_df.groupby(['bat_tm', 'batnum', 'bat', 'm.num'])

# Calculate the desired columns
batting_match_level_df = bat_grouped_df.agg(
    tr=('runs', 'sum'),
    tbf=('b.cnt', 'sum'),
    tinn=('m.num', 'nunique'),
    
    tr_pp=('f.ov.powp', lambda x: (x * df['runs']).sum()),
    tbf_pp=('f.ov.powp', 'sum'),
    tout_pp=('wkt', lambda x: (x * df['f.ov.powp']).sum()),
    
    trc_mid=('f.ov.midd', lambda x: (x * df['runs']).sum()),
    tbf_mid=('f.ov.midd', 'sum'),
    tout_mid=('wkt', lambda x: (x * df['f.ov.midd']).sum()),
    
    trc_dth=('f.ov.deat', lambda x: (x * df['runs']).sum()),
    tbf_dth=('f.ov.deat', 'sum'),
    tout_dth=('wkt', lambda x: (x * df['f.ov.deat']).sum())
)

# Reset the index so that the primary key columns become regular columns
batting_match_level_df.reset_index(inplace=True)


In [123]:

display(batting_match_level_df.head(3))


,bat_tm,batnum,bat,m.num,tr,tbf,tinn,tr_pp,tbf_pp,tout_pp,trc_mid,tbf_mid,tout_mid,trc_dth,tbf_dth,tout_dth
0,Chennai Super Kings,1,DP Conway,1,1,6,1,5,6,1,0,0,0,0,0,0
1,Chennai Super Kings,1,DP Conway,12,0,4,1,6,4,0,0,0,0,0,0,1
2,Chennai Super Kings,1,RD Gaikwad,6,57,31,1,22,21,0,11,11,1,0,0,0


In [124]:

# Add points columns to the original DataFrame
batting_match_level_df['f.cp.pts30'] = batting_match_level_df['tr'].apply(lambda x: 4 if x >= 30 else 0)
batting_match_level_df['f.cp.pts50'] = batting_match_level_df['tr'].apply(lambda x: 8 if x >= 50 else 0)
batting_match_level_df['f.cp.pts100'] = batting_match_level_df['tr'].apply(lambda x: 16 if x >= 100 else 0)
batting_match_level_df['f.cp.pts0'] = batting_match_level_df['tr'].apply(lambda x: -2 if x == 0 else 0)


In [125]:

batting_match_level_df.describe()


,batnum,m.num,tr,tbf,tinn,tr_pp,tbf_pp,tout_pp,trc_mid,tbf_mid,tout_mid,trc_dth,tbf_dth,tout_dth,f.cp.pts30,f.cp.pts50,f.cp.pts100,f.cp.pts0
count,395.000000,395.000000,395.000000,395.000000,395.0,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,4.843038,12.551899,20.255696,14.200000,1.0,5.005063,4.579747,0.177215,9.258228,7.511392,0.377215,3.716456,2.741772,0.179747,1.022785,1.093671,0.081013,-0.222785
std,2.697182,6.992325,22.033959,12.979406,0.0,8.498431,6.865687,0.382335,12.725096,9.425731,0.485304,7.523608,4.498071,0.384463,1.747220,2.751803,1.137063,0.630033
min,1.000000,1.000000,0.000000,1.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000
25%,3.000000,6.500000,3.000000,4.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,12.000000,13.000000,10.000000,1.0,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,19.000000,30.000000,21.000000,1.0,6.500000,8.000000,0.000000,15.000000,12.000000,1.000000,5.000000,4.000000,0.000000,4.000000,0.000000,0.000000,0.000000
max,11.000000,24.000000,104.000000,66.000000,1.0,46.000000,28.000000,1.000000,58.000000,40.000000,1.000000,40.000000,23.000000,1.000000,4.000000,8.000000,16.000000,0.000000


In [126]:

print(batting_match_level_df.shape)
print()
print(batting_match_level_df.columns.values.tolist())


(395, 20)

['bat_tm', 'batnum', 'bat', 'm.num', 'tr', 'tbf', 'tinn', 'tr_pp', 'tbf_pp', 'tout_pp', 'trc_mid', 'tbf_mid', 'tout_mid', 'trc_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0']


### Bowling stats

In [127]:

### DICTIONARY CREATION

# Create a dictionary with the match number ('m.num') as the key and the team names as the values
team_mapping = master_df.groupby('m.num')['bat_tm'].unique().apply(list).to_dict()

# Define a function to switch the team name
def switch_team_name(team, match_num):
    teams = team_mapping[match_num]
    return teams[0] if teams[1] == team else teams[1]

# Add a new column 'bow_team' in the bowling_match_level_df
master_df['bow_tm'] = master_df.apply(lambda x: switch_team_name(x['bat_tm'], x['m.num']), axis=1)



### ROLLING UP THE DATASET
bow_grouped_df = master_df.groupby(['bow_tm', 'bow', 'm.num'])

# Calculate the bowling stats for each player by game (m.num)
bowling_match_level_df = bow_grouped_df.agg(
    tbb=('b.cnt', 'sum'),
    tbrc=('runs', 'sum'),
    tbwkt=('wkt', 'sum'),
    tov=('over_num', 'nunique'),
    #tot_overs_powp=('over_num', lambda x: (((df['f.ov.powp'] * x)) | (x == 0)).nunique()),
    #tot_overs_mid=('over_num', lambda x: ( (df['f.ov.midd'] * x) !=0).nunique()),
    #tot_overs_dth=('over_num', lambda x: ( (df['f.ov.deat'] * x) !=0).nunique()),
    tbrc_powp=('runs', lambda x: (df['f.ov.powp'] * x).sum()),
    tbrc_mid=('runs', lambda x: (df['f.ov.midd'] * x).sum()),
    tbrc_dth=('runs', lambda x: (df['f.ov.deat'] * x).sum()),
    tbwkt_powp=('wkt', lambda x: (df['f.ov.powp'] * x).sum()),
    tbwkt_mid=('wkt', lambda x: (df['f.ov.midd'] * x).sum()),
    tbwkt_dth=('wkt', lambda x: (df['f.ov.deat'] * x).sum())
)

bowling_match_level_df.reset_index(inplace = True)

### CALCULATING STATS
"""
# Calculate the bowling stats for each player by game (m.num)
bowling_match_level_df = bow_grouped_df.agg(
    #tot_wides=('extra_type', lambda x: (x == 'wd').sum()),
    #tot_noballs=('extra_type', lambda x: (x == 'nb').sum())
    tot_balls_bowled=('b.cnt', 'sum'),
    tot_runs_conceded=('runs', 'sum'),
    tot_wickets_taken=('wkt', 'sum')
)
"""



"\n# Calculate the bowling stats for each player by game (m.num)\nbowling_match_level_df = bow_grouped_df.agg(\n    #tot_wides=('extra_type', lambda x: (x == 'wd').sum()),\n    #tot_noballs=('extra_type', lambda x: (x == 'nb').sum())\n    tot_balls_bowled=('b.cnt', 'sum'),\n    tot_runs_conceded=('runs', 'sum'),\n    tot_wickets_taken=('wkt', 'sum')\n)\n"

In [128]:

bowling_match_level_df.describe()


,m.num,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth
count,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,12.546392,19.274914,27.494845,1.054983,3.240550,8.216495,13.398625,4.546392,0.240550,0.512027,0.243986
std,6.890304,6.018938,10.913886,1.062074,0.998553,7.634170,10.258682,6.530069,0.509095,0.762726,0.568352
min,1.000000,5.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.500000,15.500000,19.000000,0.000000,3.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,24.000000,27.000000,1.000000,4.000000,7.000000,13.000000,0.000000,0.000000,0.000000,0.000000
75%,19.000000,24.000000,35.000000,2.000000,4.000000,13.000000,19.000000,8.000000,0.000000,1.000000,0.000000
max,24.000000,24.000000,69.000000,5.000000,4.000000,34.000000,49.000000,30.000000,2.000000,4.000000,4.000000


In [129]:

print(master_df.shape)

# master_df for each phase of the game
powp_df = master_df[master_df['f.ov.powp'] != 0]
mid_df = master_df[master_df['f.ov.midd'] != 0]
dth_df = master_df[master_df['f.ov.deat'] != 0]

print(powp_df.shape)
print(mid_df.shape)
print(dth_df.shape)

# Group the DataFrames by the bowler, batting team, and match number
#powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg(tot_ov_powp = ('over_num','nunique')).reset_index()
powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tov.powp'})
mid_grouped = mid_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tov.mid'})
dth_grouped = dth_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tov.dth'})


(5859, 21)
(1809, 21)
(2967, 21)
(1083, 21)


In [130]:

# Reset the index of bowling_match_level_df
#bow_grouped_df.reset_index()

bowling_match_level_df_mg = bowling_match_level_df.reset_index()

# Merge the unique overs bowled in each phase back to the bowling_match_level_df
bowling_match_level_df_mg = bowling_match_level_df_mg.merge(powp_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df_mg = bowling_match_level_df_mg.merge(mid_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df_mg = bowling_match_level_df_mg.merge(dth_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')


In [133]:

print(bowling_match_level_df_mg.columns.values.tolist())


['index', 'bow_tm', 'bow', 'm.num', 'tbb', 'tbrc', 'tbwkt', 'tov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth', 'tov.powp', 'tov.mid', 'tov.dth', 'f.cp.pts3w', 'f.cp.pts4w', 'f.cp.pts5w', 'tbecon', 'f.cp.ptsecon']


In [132]:

# BOWLING
bowling_match_level_df_mg['f.cp.pts3w'] = bowling_match_level_df_mg['tbwkt'].apply(lambda x : 4 if x >= 3 else 0 )
bowling_match_level_df_mg['f.cp.pts4w'] = bowling_match_level_df_mg['tbwkt'].apply(lambda x : 8 if x >= 4 else 0 )
bowling_match_level_df_mg['f.cp.pts5w'] = bowling_match_level_df_mg['tbwkt'].apply(lambda x : 16 if x >= 5 else 0 )

bowling_match_level_df_mg['tbecon'] = ( bowling_match_level_df_mg['tbrc'] * 6 ) / bowling_match_level_df_mg['tbb']

bowling_match_level_df_mg['f.cp.ptsecon'] = bowling_match_level_df_mg.apply(lambda x : (2*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) \
                                                            else ( (4*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) else 0 )\
                                                , axis =1)



In [134]:

display(bowling_match_level_df_mg.head(3))


,index,bow_tm,bow,m.num,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,...,tbwkt_mid,tbwkt_dth,tov.powp,tov.mid,tov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon
0,0,Chennai Super Kings,Akash Singh,17,24,39,2,4,14,15,...,1,1,2.0,1.0,1.0,0,0,0,9.750000,0.0
1,1,Chennai Super Kings,Akash Singh,24,18,35,1,3,29,0,...,0,0,3.0,NaN,NaN,0,0,0,11.666667,0.0
2,2,Chennai Super Kings,BA Stokes,6,6,18,0,1,18,0,...,0,0,1.0,NaN,NaN,0,0,0,18.000000,0.0


In [135]:

bowling_match_level_df_mg.describe()


,index,m.num,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth,tov.powp,tov.mid,tov.dth,f.cp.pts3w,f.cp.pts4w,f.cp.pts5w,tbecon,f.cp.ptsecon
count,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,291.000000,187.000000,239.000000,141.000000,291.000000,291.000000,291.000000,291.000000,291.000000
mean,145.000000,12.546392,19.274914,27.494845,1.054983,3.240550,8.216495,13.398625,4.546392,0.240550,0.512027,0.243986,1.540107,2.008368,1.241135,0.439863,0.192440,0.054983,8.899206,0.042955
std,84.148678,6.890304,6.018938,10.913886,1.062074,0.998553,7.634170,10.258682,6.530069,0.509095,0.762726,0.568352,0.641106,1.008333,0.429297,1.253543,1.227871,0.937937,3.078170,0.186126
min,0.000000,1.000000,5.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000
25%,72.500000,6.500000,15.500000,19.000000,0.000000,3.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,6.775000,0.000000
50%,145.000000,12.000000,24.000000,27.000000,1.000000,4.000000,7.000000,13.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,8.250000,0.000000
75%,217.500000,19.000000,24.000000,35.000000,2.000000,4.000000,13.000000,19.000000,8.000000,0.000000,1.000000,0.000000,2.000000,3.000000,1.000000,0.000000,0.000000,0.000000,10.500000,0.000000
max,290.000000,24.000000,24.000000,69.000000,5.000000,4.000000,34.000000,49.000000,30.000000,2.000000,4.000000,4.000000,3.000000,4.000000,2.000000,4.000000,8.000000,16.000000,20.000000,2.000000


In [ ]:

# QC
#bowling_match_level_df_mg[bowling_match_level_df_mg['tov.powp'] == 4]


## MERGE

In [136]:

### MERGE BAT AND BOW DF

# Reset the index for both DataFrames to prepare for merging
batmldf = batting_match_level_df.reset_index(drop = True)
bowmldf = bowling_match_level_df.reset_index(drop = True)

dropcols = ['bow_tm','bow']

# Merge the batting figures with the bowling stats
merged_df = pd.merge( batmldf, bowmldf, 
                     left_on=['bat_tm', 'bat', 'm.num'],
                     right_on=['bow_tm', 'bow', 'm.num'],
                     how='outer', 
                     suffixes=('_bat', '_bowl')).rename(columns = {'bat_tm':'tm', 'bat':'name' }).drop(columns = dropcols)

# Replace NaN values with 0 in the merged DataFrame
#merged_df.fillna(0, inplace=True)

print(merged_df.shape)


(562, 30)


In [137]:

print(merged_df.columns.values.tolist())


['tm', 'batnum', 'name', 'm.num', 'tr', 'tbf', 'tinn', 'tr_pp', 'tbf_pp', 'tout_pp', 'trc_mid', 'tbf_mid', 'tout_mid', 'trc_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0', 'tbb', 'tbrc', 'tbwkt', 'tov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth']


In [138]:

# Print the merged DataFrame
display(merged_df.head(3))


,tm,batnum,name,m.num,tr,tbf,tinn,tr_pp,tbf_pp,tout_pp,...,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth
0,Chennai Super Kings,1.0,DP Conway,1,1.0,6.0,1.0,5.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chennai Super Kings,1.0,DP Conway,12,0.0,4.0,1.0,6.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chennai Super Kings,1.0,RD Gaikwad,6,57.0,31.0,1.0,22.0,21.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:

merged_df.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
batnum,395.0,4.843038,2.697182,1.0,3.00,5.0,7.0,11.0
m.num,562.0,12.539146,6.967922,1.0,6.25,12.5,19.0,24.0
tr,395.0,20.255696,22.033959,0.0,3.00,13.0,30.0,104.0
tbf,395.0,14.200000,12.979406,1.0,4.00,10.0,21.0,66.0
tinn,395.0,1.000000,0.000000,1.0,1.00,1.0,1.0,1.0
tr_pp,395.0,5.005063,8.498431,0.0,0.00,0.0,6.5,46.0
tbf_pp,395.0,4.579747,6.865687,0.0,0.00,0.0,8.0,28.0
tout_pp,395.0,0.177215,0.382335,0.0,0.00,0.0,0.0,1.0
trc_mid,395.0,9.258228,12.725096,0.0,0.00,2.0,15.0,58.0
tbf_mid,395.0,7.511392,9.425731,0.0,0.00,3.0,12.0,40.0


### MAIN 

In [ ]:

merged_df.groupby(['tm','name'])

['tm', 'batnum', 'name', 'm.num', 'tr', 'tbf', 'tinn', 'tr_pp', 'tbf_pp', 'tout_pp', 'trc_mid', 'tbf_mid', 'tout_mid', 'trc_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0', 'tbb', 'tbrc', 'tbwkt', 'tov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth']


In [154]:


pstats = merged_df.groupby(['tm', 'name']).agg({
    'batnum': np.min,
    'm.num': 'nunique',
    'tr': 'sum',
    'tbf': 'sum',
    'tinn': 'sum',
    'tr_pp': 'sum',
    'tbf_pp': 'sum',
    'tout_pp': 'sum',
    'trc_mid': 'sum',
    'tbf_mid': 'sum',
    'tout_mid': 'sum',
    'trc_dth': 'sum',
    'tbf_dth': 'sum',
    'tout_dth': 'sum',
    'f.cp.pts30': 'sum',
    'f.cp.pts50': 'sum',
    'f.cp.pts100': 'sum',
    'f.cp.pts0': 'sum',
    'tbb': 'sum',
    'tbrc': 'sum',
    'tbwkt': 'sum',
    'tov': 'sum',
    'tbrc_powp': 'sum',
    'tbrc_mid': 'sum',
    'tbrc_dth': 'sum',
    'tbwkt_powp': 'sum',
    'tbwkt_mid': 'sum',
    'tbwkt_dth': 'sum'
}).reset_index()



In [155]:

print(pstats.shape)
print()
print(pstats.columns.values.tolist())
print()


(127, 30)

['tm', 'name', 'batnum', 'm.num', 'tr', 'tbf', 'tinn', 'tr_pp', 'tbf_pp', 'tout_pp', 'trc_mid', 'tbf_mid', 'tout_mid', 'trc_dth', 'tbf_dth', 'tout_dth', 'f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0', 'tbb', 'tbrc', 'tbwkt', 'tov', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth']



In [158]:

pstats.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
batnum,127.0,5.102362,3.054621,1.0,2.0,5.0,8.0,11.0
m.num,127.0,3.110236,1.559550,1.0,2.0,3.0,5.0,5.0
tr,127.0,63.000000,66.464455,0.0,10.0,37.0,93.0,259.0
tbf,127.0,44.165354,43.628075,1.0,8.0,31.0,64.0,194.0
tinn,127.0,3.110236,1.559550,1.0,2.0,3.0,5.0,5.0
tr_pp,127.0,15.566929,27.654785,0.0,0.0,0.0,17.5,127.0
tbf_pp,127.0,14.244094,24.251891,0.0,0.0,0.0,17.0,97.0
tout_pp,127.0,0.551181,1.013470,0.0,0.0,0.0,1.0,5.0
trc_mid,127.0,28.795276,33.671697,0.0,0.5,14.0,49.0,142.0
tbf_mid,127.0,23.362205,26.549047,0.0,1.0,13.0,35.5,104.0


In [156]:

display(pstats)


,tm,name,batnum,m.num,tr,tbf,tinn,tr_pp,tbf_pp,tout_pp,...,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth
0,Chennai Super Kings,AM Rahane,2.0,3,129.0,66.0,3.0,73.0,45.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Chennai Super Kings,AT Rayudu,5.0,5,74.0,50.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chennai Super Kings,BA Stokes,4.0,2,15.0,14.0,2.0,4.0,1.0,0.0,...,6.0,18.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0
3,Chennai Super Kings,DP Conway,1.0,5,181.0,121.0,5.0,44.0,58.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Chennai Super Kings,MJ Santner,9.0,2,2.0,4.0,2.0,0.0,0.0,0.0,...,48.0,53.0,1.0,8.0,9.0,44.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Sunrisers Hyderabad,HC Brook,1.0,4,129.0,93.0,4.0,73.0,50.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,Sunrisers Hyderabad,MA Agarwal,2.0,4,65.0,63.0,4.0,49.0,51.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,Sunrisers Hyderabad,RA Tripathi,2.0,4,117.0,95.0,4.0,15.0,20.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,Sunrisers Hyderabad,Umran Malik,10.0,1,19.0,7.0,1.0,0.0,0.0,0.0,...,18.0,31.0,1.0,3.0,0.0,31.0,0.0,0.0,1.0,0.0


### Fantasy Points Totals

In [ ]:

#Create Fantasy pts crickpe version

pstats['bat_pts'] = master_df['rs'].fillna(0) + master_df['four'].fillna(0) + master_df['six'].fillna(0)*2 \
    + master_df['pts_for_thrty'].fillna(0) + master_df['pts_for_fifty'].fillna(0) + master_df['pts_for_hundy'].fillna(0) + master_df['pts_for_ducky'].fillna(0)

master_df['bow_pts'] = master_df['wkt'].fillna(0)*25 \
    + master_df['pts_for_3w'].fillna(0) + master_df['pts_for_4w'].fillna(0) + master_df['pts_for_5w'].fillna(0) \
        + master_df['pts_for_bow_econ'].fillna(0)

master_df['pts_crickpe'] = master_df['bat_pts'].fillna(0) + master_df['bow_pts'].fillna(0)


In [ ]:

# Calculate strike rates for each phase of the game
#df_plyr_mtch['sr_powp'] = (df_plyr_mtch['tot_runs_powp'] / df_plyr_mtch['tot_balls_powp']) * 100
#df_plyr_mtch['sr_mid'] = (df_plyr_mtch['tot_runs_mid'] / df_plyr_mtch['tot_balls_mid']) * 100
#df_plyr_mtch['sr_dth'] = (df_plyr_mtch['tot_runs_dth'] / df_plyr_mtch['tot_balls_dth']) * 100


In [161]:

display(pstats.head(3))


,tm,name,batnum,m.num,tr,tbf,tinn,tr_pp,tbf_pp,tout_pp,...,tbb,tbrc,tbwkt,tov,tbrc_powp,tbrc_mid,tbrc_dth,tbwkt_powp,tbwkt_mid,tbwkt_dth
0,Chennai Super Kings,AM Rahane,2.0,3,129.0,66.0,3.0,73.0,45.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Chennai Super Kings,AT Rayudu,5.0,5,74.0,50.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chennai Super Kings,BA Stokes,4.0,2,15.0,14.0,2.0,4.0,1.0,0.0,...,6.0,18.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0


In [162]:

exp_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'
#exp_filename = 'pstats_firstpass.xlsx'

#master_df.to_csv( exp_location + exp_filename, index = False)
master_df.to_excel( exp_location + exp_filename, index = False)
